# 单量子比特含噪模拟器


*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要

本教程将介绍如何使用量脉的单量子比特含噪模拟器，定义退相干噪声（decoherence noise）和振幅失真噪声（amplitude noise），并通过自定义的控制脉冲实现量子门。本教程大纲如下：

- 背景介绍
- 准备工作
- 自定义模拟器参数
- 拉比振荡校准 $\pi$ 脉冲
- 测量弛豫时间 $T_1$
- 用 Ramsey 实验测量失相时间 $T_2$
- 单比特模拟器的门级控制
- 总结

## 背景介绍

在脉冲层级上，单量子比特的控制需要对物理量子比特施加控制脉冲。在本教程中，单量子比特含噪模拟器可以根据用户输入的控制脉冲和噪声参数来实现对量子比特控制的动力学演化的模拟。

一个三能级的超导量子比特，经过在本征频率下的旋转坐标系转换（rotating frame）以及旋转波近似（Rotating Wave Approximation, RWA），其系统哈密顿量可以写成 \[1\]：

$$
\hat{H}_{\rm sys}(t) = \hat{H}_{\rm anharm} + \hat{H}_{\rm noise}(t) + \hat{H}_{\rm ctrl}(t),
$$

其中，哈密顿量的非谐项 $\hat{H}_{\rm anharm}$ 与物理量子比特的非谐性 $\alpha$ 有关：

$$
\hat{H}_{\rm anharm} = \frac{\alpha}{2} \hat{a}^\dagger \hat{a}^\dagger \hat{a}\hat{a},
$$

其中 $\hat{a}^\dagger$ 以及 $\hat{a}$ 分别是三能级量子比特的产生算符和湮灭算符。在量脉的单量子模拟器中，我们特别地引入了两种噪声项：由于与环境作用的退相干噪声以及由于失真的脉冲波形的波动引起的振幅噪声 $\hat{H}_{\rm amp}$ \[1\]:

$$
\hat{H}_{\rm amp}(t) = \epsilon \hat{H}_{\rm ctrl}.
$$

其中，脉冲失真的噪声参数 $\epsilon$ 的概率分布服从高斯分布：

$$
P(\epsilon) = \frac{1}{\sqrt{ 2 \pi \sigma^2 }}e^{ - \frac{ \epsilon^2 } {2 \sigma^2} }.
$$

而与退相干相关的噪声参数 $T_1$、$T_2$ 描述了量子态的密度矩阵的非对角元的衰减速率以及激发态的布居数衰减速率。可以用 Bloch-Redfield 密度矩阵 $\rho_{BR}$ 表示初态是 $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$ 的量子比特在开放系统下的退相干演化 ($\delta \omega$ 是控制脉冲的频率和量子比特频率的差) \[2\]：

$$
\rho_{BR} = 
\begin{pmatrix} 
1 + (|\alpha|^2 - 1)e^{-t/T_1 } & \alpha\beta^* e^{i\delta \omega t}e^{-t/T_2} \\
\alpha^* \beta e^{-i\delta\omega t} e^{-t/T_2} & |\beta|^2 e^{-t/T_1}
\end{pmatrix}.
$$

从上式可以看到，激发态布居数的衰减速率与 $T_1$ 有关，密度矩阵的非对角元素的衰减速率与 $T_2$ 有关。

因此我们用与脉冲失真的概率分布有关的系数 $\sigma$，以及退相干相关的系数 $T_1$、$T_2$ 来表征这两种噪声, 即用户可以通过改变这三个参数研究所模拟系统的噪声。 


## 准备工作

成功安装量脉后，您可以按照本教程运行下面的程序。在运行此教程前，您需要从量脉（Quanlse）和其它常用的 Python 库导入以下包：

In [ ]:
# Import 1-qubit noisy simulator at the pulse level
from Quanlse.Simulator import PulseModel
from Quanlse.Simulator.PulseSim1Q import pulseSim1Q
from Quanlse.QOperator import driveX, driveZ
from Quanlse.QWaveform import gaussian, square
from Quanlse.Utils.Functions import basis, dagger, expect, project
from Quanlse.QOperation.FixedGate import H, X, Y, Z
from Quanlse.Utils.Bloch import plotBloch, rho2Coordinate
from Quanlse.remoteSimulator import remoteSimulatorRunHamiltonian as runHamiltonian

# Import tool for analysis
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
from math import pi
import numpy as np

使用量脉云服务之前，我们需要一个 token 来访问云端。

In [ ]:
# Import Define class and set the token for cloud service
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ''

## 自定义模拟器参数

我们首先定义该模拟器的必要参数以及噪声相关的参数。

In [ ]:
qubitNum = 1  # The number of qubits
dt = 0.2  # The sampling period, in nanoseconds
level = 3  # The energy level
anharm = -0.3472 * (2 * pi)  # The anharmonicity of the qubit, in 2 * pi * GHz
wq = 4.9  * (2 * pi)  # The qubit frequency, in 2 * pi * GHz

# Define the noise of the simulator
ampSigma = 0.02  # amplitude (over-rotation) error
t1 = 3000  # qubit relaxation time, in nanoseconds
t2 = 800  # qubit dephasing time, in nanoseconds

创建 `PulseModel` 类的一个对象，该对象是根据上述参数定义的一个物理模型。

In [ ]:
qubitAnharm = {0: anharm}
qubitFreq  = {0: wq}
qubitT1 = {0: t1}
qubitT2 = {0: t2}

model = PulseModel(subSysNum=qubitNum, sysLevel=level, dt=dt, ampSigma=ampSigma,
                   T1=qubitT1, T2=qubitT2, qubitFreq=qubitFreq, qubitAnharm=qubitAnharm)

ham = model.createQHamiltonian()

这样我们的模拟器的物理建模就完成了。接下来我们可以用拉比振荡实验来找到 $\pi$ 脉冲以及 $\pi / 2$ 脉冲对应的幅值。

## 拉比振荡校准 $\pi$ 脉冲

本节中我们利用拉比振荡，即固定脉冲的持续时间，不断改变脉冲幅度并且得到 $|1\rangle$ 态的布居数来进行 $\pi$ 脉冲的校准实验。我们使用单量子比特模拟器来模拟这个过程。

In [ ]:
# Define the amplitudes of the pulse
ampList = np.linspace(0, 0.4, 100)

# Create a jobList object
rabiJob = ham.createJobList()

# Define the shape of the gaussian waveform
tg = 60
tau = tg / 2
sigma = tg / 8

# Append each job of different pulse amplitudes to jobList
for amp in ampList:
    wave = gaussian(t=tg, a=amp, tau=tau, sigma=sigma)
    job = ham.createJob()
    job.appendWave(operators=driveX, onSubSys=0, waves=wave)
    job = model.getSimJob(job)
    rabiJob.addJob(job)

当定义好拉比振荡的脉冲任务后，我们将该任务以及初始态 $|\psi\rangle = |0\rangle$ 传入函数 `runHamiltonian()` 里，并且将参数 `isOpen` 设为 `True` 来模拟开放系统的时间演化。 

In [ ]:
# Define the initial state for the simulation
stateInit = basis(level, 0) 
    
# Run the simulation of the open system evolution
result = runHamiltonian(ham=ham, state0=stateInit, jobList=rabiJob, isOpen=True)

接下来我们画出不同脉冲幅度下的激发态 $|1\rangle$ 布居数。

In [ ]:
# Define the projector
prj = basis(level, 1) @ dagger(basis(level, 1))

popList = []

# Compute the population for each job
for res in result:
    rho = res['state']
    popList.append(expect(prj, rho))  # Compute the population of |1>

plt.plot(ampList, popList, '.')
plt.xlabel('Amplitudes (a.u.)')
plt.ylabel(r'Population of $|1\rangle$')
plt.title('Rabi oscillation')
plt.show()

然后我们拟合得到的点，获得对应的余弦函数。

In [ ]:
# Define the function to be fitted
def fit(x, a, b, c, d):
    return a * np.cos(b * x + c) + d

# Fit the curve
paraFit, _ = curve_fit(fit, ampList, popList, [-0.5, 2 * np.pi / 0.3, 0, 0.5])
def yFit(x):
    return fit(x, paraFit[0], paraFit[1], paraFit[2], paraFit[3])
y = [yFit(x) for x in ampList]

# Plot the fitted curve
plt.plot(ampList, y)
plt.xlabel('Amplitudes (a.u.)')
plt.ylabel(r'Population of $|1\rangle$')
plt.ylim(-0.05, 1)
plt.title('Rabi oscillation')
plt.show()

可以看到，$|1\rangle$ 的布居数随脉冲幅值的增大呈现周期性的振荡。利用得到的拟合函数，我们只需要找到当 $|1\rangle$ 的布居数分别是 0.5 和 1 的时对应的幅值作为 $\pi / 2$ 脉冲以及 $\pi$ 脉冲的幅值，并且记录下来。

In [ ]:
ampList = np.linspace(0, 0.4, 5000)

piAmp = []
halfPiAmp = []
for amp in ampList:
    if abs(yFit(amp) - 0.5) < 1e-3:
        halfPiAmp.append(amp)
    if abs(yFit(amp) - 0.98) < 1e-3:
        piAmp.append(amp)

# find the corresponding amplitudes
x90 = min(halfPiAmp)
x180 = min(piAmp)

# Print the results
print(f'The amplitudes of pi/2 pulse: {x90}')
print(f'The amplitudes of pi pulse: {x180}')   

## 测量弛豫时间 $T_1$

在实验上，欲测量某个量子比特的弛豫时间 $T_1$，首先在此量子比特上施加一个 $\pi$ 脉冲，使得激发态的布居数达到最大，然后在计算布居数随时间的演化，就能得到弛豫时间 $T_1$。

In [ ]:
# Define the idling time
tIdle = 2 * t1

# Define the wave to flip the qubit state
wave = gaussian(t=tg, a=x180, tau=tau, sigma=sigma)

# Initialize a job
job = ham.createJob()

# Firstly, apply a X gate to flip the qubit
job.appendWave(operators=driveX, onSubSys=0, waves=wave)

# Then simulate the evolution during the idling time
job.appendWave(operators=driveX, onSubSys=0, waves=square(tIdle, 0))

定义好脉冲任务以及初始态后，我们开始模拟系统的含时演化。

In [ ]:
# Define the initial state for the simulation
stateInit = basis(level, 0) 
    
# Run the simulation of the open system evolution
result = runHamiltonian(ham=ham, state0=stateInit, job=job, isOpen=True)

# Define the projector |1><1|
prj = basis(level, 1) @ dagger(basis(level, 1))

popList = []

# Calculate the population of |1> during the evolution
for rho in result[0]['evolution_history']:
    popList.append(expect(prj, rho))

# Get the maximum time of the job
maxTime, _ = job.computeMaxTime()

tList = np.linspace(0, maxTime, len(popList))

# Plot the time-evolution poplulation for simulation and prediction 
plt.plot(tList, popList, '-', label='simulation')
tIdleList = np.linspace(tg, tIdle, 20)
plt.plot(tIdleList, np.exp(-1. / t1 * np.array(tIdleList - tg)), label='prediction')
plt.xlabel('Time (ns)')
plt.ylabel(r'Population of $|1\rangle$')
plt.title(r'$T_1$ measurement')
plt.legend()
plt.show()

上图是激发态布居数随时间的变化图，可见从实验刚开始的几十纳秒内，施加的 $X$ 门的脉冲使得量子比特从基态激发到激发态。然后在后面的空转时间内，激发态布居数的衰减速率与理论值吻合。可以看到在时间 $t=T_1$ 秒的时候，$|1\rangle$ 布居数大约是 $1/e$。

## 用 Ramsey 实验测量失相时间 $T_2$

在实验上，欲测量某个量子比特的失相时间 $T_2$，首先在此量子比特上施加一个 $\pi/2$ 脉冲，然后再经过一段空转时间 $t_{\rm idle}$ 后再施加 $\pi/2$ 脉冲，得到 $|1\rangle$ 的布居数。在这个过程中，因为驱动的频率与量子比特频率不同，所以可以观测到布居数随空转时间周期性振荡的现象。

In [ ]:
# Define the maximum idling time
maxTime = t2

# Define the detuning 
detuning = 2 * pi * 8. / maxTime

# Define the job for Ramsey experiment
tList = np.linspace(0, maxTime, 200)
ramseyJob = ham.createJobList()

for t in tList:
    job = ham.createJob()
    job.appendWave(driveX, 0, gaussian(t=tg, a=x90, tau=tau, sigma=sigma), compact=False)  # pi/2 pulse
    job.appendWave(driveZ, 0, waves=square(t, detuning), compact=False)  # simulate the rotation due to the detuning
    job.appendWave(driveX, 0, gaussian(t=tg, a=-x90, tau=tau, sigma=sigma), compact=False)  # pi/2 pulse
    ramseyJob.addJob(job)

定义好脉冲任务以及初始态后，我们开始模拟系统的含时演化。

In [ ]:
# Run the simulation starting with initial state |0>.
stateInit = basis(level, 0)
result = runHamiltonian(ham, stateInit, jobList=ramseyJob, isOpen=True)

绘制布居数随空转时间的变化图。

In [ ]:
popList = []

prj = basis(level, 1) @ dagger(basis(level, 1))

for res in result:
    rho = res['state']
    popList.append(expect(prj, rho))

plt.plot(tList, popList, '.b', label='simulation')
plt.xlabel('Delayed time (ns)')
plt.ylabel(r'Population of $|1\rangle$')
plt.legend()
plt.show()

然后我们用拟合函数拟合该曲线，并且根据拟合函数的参数估计 $T_2$。

In [ ]:
# Define the fitting function
def fitRamsey(x, a, b):
    return - np.cos(a * x) * np.exp(- b * x) * 0.5 + 0.5

paraFit, _ = curve_fit(fitRamsey, tList, popList, [detuning, 0.])

def yFit(x):
    return fitRamsey(x, paraFit[0], paraFit[1])

# Plot the result
plt.plot(tList, popList, '.b', label='simulation')
plt.plot(tList, yFit(tList), label='fit')
plt.plot(tList, np.exp(- (1 / t2 + 1/(2 * t1)) * tList) * 0.5 + 0.5, label='prediction')
plt.xlabel('Delayed time (ns)')
plt.ylabel(r'Population of $|1\rangle$')
plt.legend()
plt.show()

# Print the estimated T_2 time
print(f'The T2-dephasing time is approximately {1 / (paraFit[1] - 1 / (2 * t1))} ns')

## 单比特含噪模拟器的门级控制

除了从脉冲层模拟单量子比特的操作外，我们也可以直接从编译好的逻辑门层面直接使用该模拟器。
首先，我们直接调用一个已经预设好的单量子比特的 `PulseSim1Q()` 的对象.

In [ ]:
model = pulseSim1Q(dt=0.2)
ham = model.createQHamiltonian()

然后对该量子比特设置一系列不同的门操作。这里，我们首先定义一组单量子比特门。

In [ ]:
H(model.Q[0])
X(model.Q[0])
Y(model.Q[0])
Z(model.Q[0])
H(model.Q[0])

调用方法 `model.schedule()` 来生成定义好的门的脉冲序列，然后调用 `runHamiltonian` 模拟开放系统下单量子比特在退相干情况下的时间演化。该演化可以通过调用绘制量子态在布洛赫球上的演化的函数 `plotBloch()` 来可视化。

In [ ]:
job = model.schedule()

res = runHamiltonian(ham, state0=basis(3, 0), job=job, isOpen=True)

history = res[0]['evolution_history']

posList = []

for rho in history:
    rho2d = project(rho, 1, 3, 2) / np.trace(rho)
    posList.append(rho2Coordinate(rho2d))
    
plotBloch(posList, save=True, mode='animate')

可以看到，在退相干比较严重的情况下，量子态的布洛赫矢量无法保持在球面上——也就是说量子态从纯态演化成混合态，密度矩阵的非对角元衰减，系统与外界作用的时候发生信息的损失。因此当量子比特的 $T_1$ 以及 $T_2$ 较短的情况下，由退相干带来的噪声不利于深度比较大的量子电路。

## 总结

本教程介绍了如何使用量脉对考虑部分噪声的超导单量子比特的控制进行模拟，并可视化结果。用户可以点击这个链接 [tutorial-single-qubit-noisy-simulator.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/CN/tutorial-single-qubit-noisy-simulator-cn.ipynb) 跳转到此 Jupyter Notebook 文档相应的 GitHub 页面来获取相关代码，并尝试不同于本教程的参数值来进一步探索量脉单量子比特含噪模拟器模块的功能。

## 参考文献

\[1\] [Carvalho, Andre RR, et al. "Error-robust quantum logic optimization using a cloud quantum computer interface." *arXiv preprint arXiv:2010.08057* (2020).](https://arxiv.org/abs/2010.08057)

\[2\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://aip.scitation.org/doi/abs/10.1063/1.5089550)